In [1]:
# external api
import fundamentalanalysis as fa
import yfinance as yf

# data analytic 
import numpy as np
import pandas as pd

# additional liberies
import datetime
import statistics as st

# визуализация
import matplotlib.pyplot as plt
from matplotlib.pyplot import figure
import seaborn as sns
import warnings
warnings.simplefilter('ignore')
%matplotlib inline 

# ml 
from sklearn.naive_bayes import GaussianNB
from sklearn.model_selection import train_test_split


In [2]:
pd.set_option('display.max_rows', 1000)
pd.set_option('display.max_columns', 500)
pd.set_option('display.width', 1000)

In [3]:
# api_key = '00ef9804fcde0edd93b1b4821ee2f06a' #job.chap@icloud.com
api_key = 'c81352430e2fe3c941faf0814227562b' #jobs.chaps@gmail.com
# api_key = 'a765d11740cccfb61177da0ad8699d1e' #job.chap@gmail.com

In [4]:
''' Колонки '''
# ticker = 'ATEX'
# ticker = 'F'
# tickers_list = ['AAPL', 'ADBE', 'COP', 'AMAT', 'MFGP', 'FISV', 'ASAN', 'ALIT', 'DAVA', 'NCR', 'SMTC', 'EXLS', 'JKHY', 'ORCL', 'APPS', 'U']
tickers_list = ['AAPL' ,'ADBE', 'COP', 'AMAT', 'MFGP', 'FISV', 'ASAN', 'ALIT', 'DAVA', 'NCR', 'SMTC', 'EXLS', 'JKHY', 'ORCL', 'APPS', 'U', 'FICO', 'WEX', 'TXN', 'GDDY', 'CRSR', 'MANH', 'LITE', 'MANH', 'TENB']

''' для фильтрации'''
filt_start_date = '2000'
filt_finish_date = '2022'


''' время '''
stime=f'{str(int(filt_start_date)-1)}-01-01'
ftime='2023-02-01'
period = '1mo'  # '1d' '1wk'

In [5]:
def get_tick_table(tick, stime, period):
    try:
        def get_data_from_ticker(tick, stime, ftime, interval=period):
            ticker = yf.Ticker(tick)

            df = ticker.history(start=stime, end=ftime, interval=interval)
            x = pd.DataFrame(df)
            x.rename(columns={"Close": tick}, inplace=True)
            z = x.drop(columns=["Open", "High", "Low", "Volume", "Dividends", "Stock Splits"])
            return z

        df = get_data_from_ticker(tick, stime, None, period)
        df_market = get_data_from_ticker('^GSPC', stime, None, period)


        def change_index(df):
            df_index = df.index.to_list()
            for i in range(len(df_index)):
                t = df_index[i].date()
                df_index[i] = t.strftime('%Y-%m-%d')
            df['Date_index'] = df_index
            df = df.set_index('Date_index')
            return df

        df = change_index(df)
        df_market  = change_index(df_market)


        def change_index_plus_one(df):
            ''' добавляем цифру к году, чтобы видень данные как начало года, а не как факт 
            '''
            df_index = df.index.to_list()
            for i in range(len(df_index)):
                df_index[i] = str(int(df_index[i])+1)
            df['Date_index'] = df_index
            df = df.set_index('Date_index')
            return df


        key_metrics = fa.key_metrics(tick, api_key, period='annual')
        df_key_metr = key_metrics.T
        df_key_metr = change_index_plus_one(df_key_metr)

        df_key_metr = df_key_metr.loc[filt_finish_date:filt_start_date].copy(deep=True)
        index_years = df_key_metr.index.to_list()


        def get_price_from_df(df, list_dates):
            new_list = []
            for i in range(len(list_dates)):
                x = int(list_dates[i])
                # x += 1
                x = str(x)
                r = df.loc[f'{x}-01-01']
                r = float(r)
                new_list.append(r)

            return new_list


        stock_year_prices = get_price_from_df(df, index_years)
        stock_year_market_prices = get_price_from_df(df_market, index_years)


        df_key_metr['stock'] = list(stock_year_prices)
        df_key_metr['SNP'] = list(stock_year_market_prices)
        df_key_metr.drop('period', axis=1, inplace=True)


        def change_percent_all_columns(df):
            x = df.columns.to_list()
            for i in range(len(x)):
                v = df[x[i]].to_list()
                l = []
                for y in range(len(v)):
                    try:
                        # if y != len(v):
                        #     vv = (v[y] / (v[y+1] / 100)) - 100
                        #     l.append(vv)
                        #     # print(vv)
                        if y != len(v):
                            if v[y] < 0 and v[y+1] > 0:
                                l.append('Minus')
                            elif v[y] > 0 and v[y+1] < 0:
                                l.append('Plus')
                            else:
                                vv = (v[y] / (v[y+1] / 100)) - 100
                                l.append(vv)  
                        else:
                            vv = 0
                            l.append(vv)
                            # print(vv)
                    except:
                        l.append(0)
                df[f'change % {x[i]}'] = l
            return df

        df_with_changes_key = change_percent_all_columns(df_key_metr)


        df_with_changes_key['ticker_name'] = tick
        
        # удаляем последнюю строку, где значения равны 0
        df_with_changes_key = df_with_changes_key.drop(df_with_changes_key.index[-1])
        
        ''' приводим результаты к 1 (рост за год) и 0 (падение)'''

        t = []
        for i in range(len(df_with_changes_key[f'change % stock'])):
            if df_with_changes_key[f'change % stock'][i] > 0:
                t.append(1)
            else:
                t.append(0)

        df_with_changes_key['Result'] = t
        print(tick)
        
    except:
        df_with_changes_key = 0
        
    return df_with_changes_key




In [6]:
def conc_tables(tickers_list, stime, period):
    ''' загружаем список тикеров, получаем обработанную таблицу с объединенными тикерами
    '''
    try:
        for i in range(len(tickers_list)):
            if i == 0:
                df = get_tick_table(tickers_list[i], stime, period)
            elif tickers_list[i] == 0:
                pass
            else:
                df = pd.concat([df, get_tick_table(tickers_list[i], stime, period)], ignore_index=True)
    except:
        df = 0
                
    return df


df_conc_tickers = conc_tables(tickers_list, stime, period)

AAPL
ADBE
COP
AMAT
MFGP
FISV
ASAN
ALIT
DAVA
NCR
SMTC
EXLS
JKHY
ORCL
APPS
U
FICO
WEX
TXN
GDDY
CRSR
MANH
LITE
MANH
TENB


In [7]:
df_conc_tickers.head()

,revenuePerShare,netIncomePerShare,operatingCashFlowPerShare,freeCashFlowPerShare,cashPerShare,bookValuePerShare,tangibleBookValuePerShare,shareholdersEquityPerShare,interestDebtPerShare,marketCap,enterpriseValue,peRatio,priceToSalesRatio,pocfratio,pfcfRatio,pbRatio,ptbRatio,evToSales,enterpriseValueOverEBITDA,evToOperatingCashFlow,evToFreeCashFlow,earningsYield,freeCashFlowYield,debtToEquity,debtToAssets,netDebtToEBITDA,currentRatio,interestCoverage,incomeQuality,dividendYield,payoutRatio,salesGeneralAndAdministrativeToRevenue,researchAndDdevelopementToRevenue,intangiblesToTotalAssets,capexToOperatingCashFlow,capexToRevenue,capexToDepreciation,stockBasedCompensationToRevenue,grahamNumber,roic,returnOnTangibleAssets,grahamNetNet,workingCapital,tangibleAssetValue,netCurrentAssetValue,investedCapital,averageReceivables,averagePayables,averageInventory,daysSalesOutstanding,daysPayablesOutstanding,daysOfInventoryOnHand,receivablesTurnover,payablesTurnover,inventoryTurnover,roe,capexPerShare,stock,SNP,change % revenuePerShare,change % netIncomePerShare,change % operatingCashFlowPerShare,change % freeCashFlowPerShare,change % cashPerShare,change % bookValuePerShare,change % tangibleBookValuePerShare,change % shareholdersEquityPerShare,change % interestDebtPerShare,change % marketCap,change % enterpriseValue,change % peRatio,change % priceToSalesRatio,change % pocfratio,change % pfcfRatio,change % pbRatio,change % ptbRatio,change % evToSales,change % enterpriseValueOverEBITDA,change % evToOperatingCashFlow,change % evToFreeCashFlow,change % earningsYield,change % freeCashFlowYield,change % debtToEquity,change % debtToAssets,change % netDebtToEBITDA,change % currentRatio,change % interestCoverage,change % incomeQuality,change % dividendYield,change % payoutRatio,change % salesGeneralAndAdministrativeToRevenue,change % researchAndDdevelopementToRevenue,change % intangiblesToTotalAssets,change % capexToOperatingCashFlow,change % capexToRevenue,change % capexToDepreciation,change % stockBasedCompensationToRevenue,change % grahamNumber,change % roic,change % returnOnTangibleAssets,change % grahamNetNet,change % workingCapital,change % tangibleAssetValue,change % netCurrentAssetValue,change % investedCapital,change % averageReceivables,change % averagePayables,change % averageInventory,change % daysSalesOutstanding,change % daysPayablesOutstanding,change % daysOfInventoryOnHand,change % receivablesTurnover,change % payablesTurnover,change % inventoryTurnover,change % roe,change % capexPerShare,change % stock,change % SNP,ticker_name,Result
0,21.903541,5.669029,6.229346,5.565624,3.750553,3.777557,3.777557,3.777557,7.626006,2482477060059.236816,2572256060059.236816,26.219656,6.786117,23.861253,26.706799,39.348186,39.348186,7.031538,20.889554,24.724198,27.672652,0.038139,0.037444,1.729371,0.820257,0.729104,1.074553,41.190548,1.098838,0.005828,0.152799,0,0.059904,0,-0.106548,-0.030302,-0.982364,0.021612,21.950838,0.483099,0.269742,-10.978415,9355000000,None,-153076000000,1.976843,51506000000,54763000000,6580000000,51.390969,93.851071,11.276593,7.102415,3.88914,32.367933,1.500713,-0.663722,173.506546,4515.549805,38.452490,71.342897,33.986602,31.636843,-28.438653,0.320806,0.320806,0.320806,14.758920,24.350087,24.216691,-24.597985,-6.685682,-3.575435,-1.854226,28.782855,28.782855,-6.785784,-18.268936,-3.678874,-1.959512,32.62245,1.889257,14.521932,2.754812,-20.623436,-21.197594,78.524688,-21.802068,-17.377394,-37.70089,0.0,-12.304712,0.0,17.603252,13.809871,48.597906,-13.123585,31.107809,59.237311,52.176785,38.552872,-75.587798,0.0,33.299662,14.878615,15.807579,12.844765,23.672587,3.220554,3.078459,28.995004,-3.120070,-2.986521,-22.477618,70.794977,57.572601,33.265729,21.573991,AAPL,1.0
1,15.820258,3.308587,4.64923,4.228014,5.241031,3.765477,3.765477,3.765477,6.64524,1996361344741.568848,2070781344741.568848,34.773151,7.272322,24.746031,27.211359,30.553901,30.553901,7.543418,25.558891,25.66851,28.225739,0.028758,0.036749,1.510078,0.798267,0.91853

In [8]:
''' обратный порядок таблицы ''' 
df_flipped = df_conc_tickers.iloc[::-1]

ML Gaussian NB

In [9]:
''' добавляем данные для обучения '''
X_data = df_conc_tickers[['revenuePerShare','change % debtToAssets', 'freeCashFlowPerShare']]  # признаки
y_data = df_conc_tickers[['Result']]  # целевая величина

In [30]:
Xtrain, Xtest, ytrain, ytest = train_test_split(X_data, y_data, random_state = 0)

In [31]:
ytest

,Result
6,0.0
140,0.0
301,0.0
220,1.0
90,1.0
229,1.0
114,1.0
60,1.0
37,0.0
295,0.0


In [11]:
model = GaussianNB()

In [12]:
model.fit(Xtrain, ytrain)

GaussianNB()

In [13]:
y_model = model.predict(Xtest)

In [28]:
y_model 

array([1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 0., 1., 1.,
       1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1.,
       1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1.,
       1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 0., 1.,
       1., 1., 1., 1., 1., 0., 1., 0., 1., 1., 1., 1., 1., 1., 1., 1., 1.,
       1., 1., 1., 1.])

In [14]:
from sklearn.metrics import accuracy_score

In [15]:
accuracy_score(ytest, y_model)

0.7415730337078652

In [20]:
from sklearn.metrics import confusion_matrix

In [23]:
confusion_matrix(ytest, y_model)


array([[ 2, 21],
       [ 2, 64]])

In [27]:
from sklearn.metrics import classification_report


In [26]:
classification_report(ytest, y_model)

'              precision    recall  f1-score   support\n\n         0.0       0.50      0.09      0.15        23\n         1.0       0.75      0.97      0.85        66\n\n    accuracy                           0.74        89\n   macro avg       0.63      0.53      0.50        89\nweighted avg       0.69      0.74      0.67        89\n'

In [36]:
xtraing

NameError: name 'xtraing' is not defined

In [39]:
Xtest

,revenuePerShare,change % debtToAssets,freeCashFlowPerShare
6,10.15548,13.539227,3.032022
140,9.505562,-7.693951,2.523582
301,20.273643,-13.789876,0.098149
220,1.639774,3.182821,0.313639
90,24.584848,4.339468,4.354545
229,1.219467,-1.418382,-0.265107
114,6.569197,36.435153,0.568345
60,131.597399,-6.087320,4.311845
37,4.337347,-42.397016,1.421455
295,22.725215,-1.302775,3.431086


In [37]:
ytest

,Result
6,0.0
140,0.0
301,0.0
220,1.0
90,1.0
229,1.0
114,1.0
60,1.0
37,0.0
295,0.0
